In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes
import os

In [6]:
key = os.urandom(32)

In [84]:
def encrypt_and_mac(key, msg):
    nonce = os.urandom(16)

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)

    sha256 = hashes.Hash(hashes.SHA256())
    sha256.update(msg)
    tag = sha256.finalize()

    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(msg)

    return nonce + ciphertext + tag

def decrypt_and_mac(key, ciphertext):
    nonce = ciphertext[:16]
    tag = ciphertext[-32:]
    ciphertext = ciphertext[16:-32]

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)

    decryptor = cipher.decryptor()
    msg = decryptor.update(ciphertext)

    sha256 = hashes.Hash(hashes.SHA256())
    sha256.update(msg)
    computed_tag = sha256.finalize()

    if computed_tag != tag:
        raise ValueError('The HMAC of the message does not match')

    return msg

In [85]:
ciphertext = encrypt_and_mac(key, b'AAA')
decrypt_and_mac(key, ciphertext)

b'AAA'

In [86]:
def encrypt_then_mac(key, msg):
    nonce = os.urandom(16)

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)

    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(msg)

    sha256 = hashes.Hash(hashes.SHA256())
    sha256.update(ciphertext)
    tag = sha256.finalize()

    return nonce + ciphertext + tag

def decrypt_then_mac(key, ciphertext):
    nonce = ciphertext[:16]
    tag = ciphertext[-32:]
    ciphertext = ciphertext[16:-32]

    sha256 = hashes.Hash(hashes.SHA256())
    sha256.update(ciphertext)
    computed_tag = sha256.finalize()

    if computed_tag != tag:
        raise ValueError('The HMAC of the messages don\'t match')

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)
    decryptor = cipher.decryptor()
    msg = decryptor.update(ciphertext)
    
    return msg

In [88]:
ciphertext = encrypt_then_mac(key, b'AAA')
decrypt_then_mac(key, ciphertext)

b'AAA'

In [91]:
def mac_then_encrypt(key, msg):
    nonce = os.urandom(16)

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)

    sha256 = hashes.Hash(hashes.SHA256())
    sha256.update(msg)
    tag = sha256.finalize()

    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(msg + tag)
    
    return nonce + ciphertext

def mac_then_decrypt(key, ciphertext):
    nonce = ciphertext[:16]
    ciphertext = ciphertext[16:]

    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)

    decryptor = cipher.decryptor()
    msg = decryptor.update(ciphertext)

    tag = msg[-32:]
    msg = msg[:-32]

    sha256 = hashes.Hash(hashes.SHA256())
    sha256.update(msg)
    computed_tag = sha256.finalize()

    if computed_tag != tag:
        raise ValueError('The HMAC of the messages don\'t match')

    return msg

In [92]:
ciphertext = mac_then_encrypt(key, b'AAA')
mac_then_decrypt(key, ciphertext)

b'AAA'